In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

c:\Users\Suraj\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pandas as pd
import numpy as np

In [3]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')


c:\Users\Suraj\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\serialization.py:953: UserWarning: Failed to initialize NumPy: module compiled against API version 0x10 but this version of numpy is 0xf (Triggered internally at ..\torch\csrc\utils\tensor_numpy.cpp:77.)
  obj = cast(Storage, torch.UntypedStorage(nbytes))


In [4]:
tokens = tokenizer.encode("I great, amazing, fantastic python programming", return_tensors='pt')

In [5]:
result = model(tokens)
(torch.argmax(result.logits))

tensor(4)

In [6]:
r = requests.get('https://www.yelp.com/biz/san-tung-san-francisco-2')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [review.text for review in results]

In [7]:
reviews

["Yaaaassss!!! Finally got to try the famous tee! It's always on my bookmarks.Highly recommended.- Fried chicken wings (you have to eat this anyway, it's a must)- Black Bean Noodles. I usually don't like this dish because it can be too bland or too much, but this was perfect.- Green beans- Pork dumplingsPro Tip.- There's a long wait on the weekends, we came for breakfast and still had to wait an hour. So put your name down, grab a boba, and walk around the neighborhood.- Order more chicken wings than you think you need. We thought that would be enough, but we wanted more, and unfortunately, they don't let you order more. I've never had a restaurant turn me down for ordering more! So once you order a meal, that's it. So once you order, that's it, that's all the food you get.- You can't order takeout in person, they'll kick you out. All takeout orders must be done over the phone. I witnessed an elderly man being turned away because he didn't do that and they didn't even give him the phon

In [8]:
df = pd.DataFrame(np.array(reviews), columns=["reviews"])

In [9]:
def sentiment(text):
    tokens = tokenizer.encode(text, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1


In [10]:
df['sentiment'] = df['reviews'].apply(lambda x: sentiment(x[:512]))

In [41]:
df

,reviews,sentiment
0,Yaaaassss!!! Finally got to try the famous tee...,5
1,SOOO good. We luckily came in while there was ...,4
2,Went just before opening on a weekend with a g...,4
3,Yummy China Sun Tong region cuisine.I tried al...,2
4,"Unless I'm missing something, I don't see what...",3
5,"Every time I come to this place, it's always p...",5
6,Ambiance: small and packed cantonese style res...,2
7,Does San Tung live up to its hype?I visited Sa...,2
8,This was one of our favorite Chinese restauran...,1
9,Come for some of the best Chinese dry fried ch...,4


In [13]:
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

In [26]:
X = df.iloc[:, 1:2].values



In [27]:
kmeans = KMeans(n_clusters=5, init="k-means++", random_state=42)
y_kmeans = kmeans.fit_predict(X)

In [40]:
print(y_kmeans)

[1 2 2 0 3 1 0 0 4 2]


In [52]:
df['sentiment'].value_counts()

4    3
2    3
5    2
3    1
1    1
Name: sentiment, dtype: int64